In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time

# Configurações visuais
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]

# Carregar os dados
df = pd.read_csv('../data/processed/base_completa_futebol.csv')

# Ver o que temos
print(f"Total de jogos: {len(df)}")
display(df.head())

Total de jogos: 1506


,match_id,league_id,league_name,season,date,round,home_team,away_team,home_goals,away_goals,status,winner,venue
0,1006026,71,Brasileirao Serie A,2023,2023-12-07T00:30:00+00:00,Regular Season - 38,Coritiba,Corinthians,0,2,FT,Away,Estádio Major Antônio Couto Pereira
1,1006028,71,Brasileirao Serie A,2023,2023-12-07T00:30:00+00:00,Regular Season - 38,Cuiaba,Atletico Paranaense,3,0,FT,Home,Arena Pantanal
2,1006020,71,Brasileirao Serie A,2023,2023-12-07T00:30:00+00:00,Regular Season - 38,Vasco DA Gama,RB Bragantino,2,1,FT,Home,Estádio São Januário
3,1006022,71,Brasileirao Serie A,2023,2023-12-07T00:30:00+00:00,Regular Season - 38,Santos,Fortaleza EC,1,2,FT,Away,Estádio Urbano Caldeira
4,1006021,71,Brasileirao Serie A,2023,2023-12-07T00:30:00+00:00,Regular Season - 38,Sao Paulo,Flamengo,1,0,FT,Home,Estádio Cícero Pompeu de Toledo


In [37]:
import pandas as pd
import numpy as np
import random

# --- 1. PREPARAÇÃO DA BASE ---
# Garante que temos o df e que está ordenado por data
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')

print(f"🔄 Gerando dados sintéticos para TODOS os {len(df)} jogos (Modo Sem Limites)...")

# --- 2. GERADOR DE ESTATÍSTICAS (O "Simulador de Realidade") ---
def gerar_stats_fake(row):
    # Lógica: Quem ganha geralmente tem stats melhores
    winner = row['winner']
    home = row['home_team']
    away = row['away_team']
    
    # Valores base
    h_shots = random.randint(3, 10)
    a_shots = random.randint(3, 10)
    h_poss = random.randint(40, 60)
    
    # Boost para o vencedor
    if winner == home:
        h_shots += random.randint(2, 6)
        h_poss += random.randint(5, 15)
    elif winner == away:
        a_shots += random.randint(2, 6)
        h_poss -= random.randint(5, 15)
        
    # Limites físicos
    if h_poss > 90: h_poss = 90
    if h_poss < 10: h_poss = 10
    a_poss = 100 - h_poss
    
    return pd.Series({
        'home_Shots on Goal': float(h_shots),
        'home_Ball Possession': float(h_poss),
        'home_Corner Kicks': float(random.randint(2, 12)),
        'home_Total Shots': float(h_shots + random.randint(3, 7)),
        'away_Shots on Goal': float(a_shots),
        'away_Ball Possession': float(a_poss),
        'away_Corner Kicks': float(random.randint(2, 12)),
        'away_Total Shots': float(a_shots + random.randint(3, 7)),
    })

# Aplica a simulação na base inteira
cols_fake = df.apply(gerar_stats_fake, axis=1)
df_completo = pd.concat([df, cols_fake], axis=1)

# --- 3. ENGENHARIA DE FEATURES (CRIANDO A "INTELIGÊNCIA") ---
print("🧠 Calculando Médias Móveis (Forma dos times nos últimos 5 jogos)...")

def calcular_medias_moveis(df_input, janela=5):
    # Organiza em formato longo (Time | Data | Stats)
    cols_home = ['date', 'match_id', 'home_team', 'home_goals', 'home_Shots on Goal', 'home_Ball Possession', 'home_Corner Kicks']
    df_home = df_input[cols_home].rename(columns={'home_team': 'team', 'home_goals': 'goals', 'home_Shots on Goal': 'shots_on_goal', 'home_Ball Possession': 'possession', 'home_Corner Kicks': 'corners'})
    
    cols_away = ['date', 'match_id', 'away_team', 'away_goals', 'away_Shots on Goal', 'away_Ball Possession', 'away_Corner Kicks']
    df_away = df_input[cols_away].rename(columns={'away_team': 'team', 'away_goals': 'goals', 'away_Shots on Goal': 'shots_on_goal', 'away_Ball Possession': 'possession', 'away_Corner Kicks': 'corners'})
    
    df_long = pd.concat([df_home, df_away]).sort_values(['team', 'date'])
    
    # O PULO DO GATO: shift(1) para não ver o futuro. Pegamos a média dos 5 jogos ANTERIORES.
    cols_num = ['goals', 'shots_on_goal', 'possession', 'corners']
    df_long_grouped = df_long.groupby('team')[cols_num].transform(lambda x: x.shift(1).rolling(window=janela, min_periods=1).mean())
    df_long_grouped.columns = [f'media_{c}' for c in cols_num]
    
    return pd.concat([df_long[['match_id', 'team']], df_long_grouped], axis=1)

# Calcula as médias
df_medias = calcular_medias_moveis(df_completo)

# --- 4. MERGE FINAL (JUNTANDO TUDO) ---
# Traz as médias de volta para a tabela principal (Home e Away)
df_medias_home = df_medias.rename(columns={'team': 'home_team'})
df_medias_home.columns = [f'home_{c}' if 'media' in c else c for c in df_medias_home.columns]

df_medias_away = df_medias.rename(columns={'team': 'away_team'})
df_medias_away.columns = [f'away_{c}' if 'media' in c else c for c in df_medias_away.columns]

# Merge
df_ia = pd.merge(df_completo, df_medias_home[['match_id', 'home_team', 'home_media_goals', 'home_media_shots_on_goal', 'home_media_possession', 'home_media_corners']], on=['match_id', 'home_team'], how='left')
df_ia = pd.merge(df_ia, df_medias_away[['match_id', 'away_team', 'away_media_goals', 'away_media_shots_on_goal', 'away_media_possession', 'away_media_corners']], on=['match_id', 'away_team'], how='left')

# Remove os primeiros jogos de cada time (que ficam sem média pois não tem histórico)
df_ia = df_ia.dropna(subset=['home_media_goals', 'away_media_goals'])

# Salva para garantir
df_ia.to_csv('../data/processed/base_pronta_para_ia.csv', index=False)

print(f"\n🚀 SUCESSO! Base pronta para Machine Learning.")
print(f"Total de jogos válidos para treino: {len(df_ia)}")
print("Exemplo de dado que a IA vai ler (Médias dos últimos 5 jogos):")
display(df_ia[['date', 'home_team', 'away_team', 'home_media_goals', 'home_media_shots_on_goal', 'winner']].tail())

🔄 Gerando dados sintéticos para TODOS os 1506 jogos (Modo Sem Limites)...
🧠 Calculando Médias Móveis (Forma dos times nos últimos 5 jogos)...

🚀 SUCESSO! Base pronta para Machine Learning.
Total de jogos válidos para treino: 1408
Exemplo de dado que a IA vai ler (Médias dos últimos 5 jogos):


,date,home_team,away_team,home_media_goals,home_media_shots_on_goal,winner
1501,2023-12-07 00:30:00+00:00,Sao Paulo,Flamengo,0.4,6.2,Home
1502,2023-12-07 00:30:00+00:00,Santos,Fortaleza EC,0.4,7.6,Away
1503,2023-12-07 00:30:00+00:00,Vasco DA Gama,RB Bragantino,1.2,7.2,Home
1504,2023-12-07 00:30:00+00:00,Cuiaba,Atletico Paranaense,1.2,7.4,Home
1505,2023-12-07 00:30:00+00:00,Coritiba,Corinthians,1.2,7.0,Away


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

print("🧠 Treinando modelo com os dados sintéticos...")

# 1. Preparar o Alvo (Target)
# Converter 'winner' (Home, Away, Draw) para números (0, 1, 2)
# Precisamos garantir que a ordem seja alfabética para bater com o modelo: Away(0), Draw(1), Home(2)
# Ou melhor, vamos fazer manual para ter certeza absoluta:
def codificar_vencedor(row):
    if row['winner'] == row['home_team']: return 2 # Home Win
    if row['winner'] == row['away_team']: return 0 # Away Win
    return 1 # Draw

df_ia['target'] = df_ia.apply(codificar_vencedor, axis=1)

# 2. Definir colunas de treino (Só as médias!)
features = [
    'home_media_goals', 'home_media_shots_on_goal', 'home_media_possession', 'home_media_corners',
    'away_media_goals', 'away_media_shots_on_goal', 'away_media_possession', 'away_media_corners'
]


# 3. Treinar
X = df_ia[features]
y = df_ia['target']

modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X, y)

print("✅ Modelo treinado na memória!")

🧠 Treinando modelo com os dados sintéticos...
✅ Modelo treinado na memória!


In [39]:
import joblib
import os

# Caminhos baseados na sua estrutura de pastas (estamos em 'notebooks/')
caminho_modelo = '../models/modelo_futebol.pkl'
caminho_dados_site = '../data/processed/base_dados_site.csv'

# 1. Salvar o Modelo
joblib.dump(modelo, caminho_modelo)
print(f"💾 Modelo salvo em: {caminho_modelo}")

# 2. Salvar a Base para o Site (apenas colunas necessárias)
# O site precisa buscar as últimas médias dos times
cols_necessarias = ['date', 'home_team', 'away_team'] + features
df_ia[cols_necessarias].to_csv(caminho_dados_site, index=False)
print(f"📦 Base de dados do site salva em: {caminho_dados_site}")

💾 Modelo salvo em: ../models/modelo_futebol.pkl
📦 Base de dados do site salva em: ../data/processed/base_dados_site.csv


In [41]:
import numpy as np
import joblib

print("🔧 CONSERTANDO A BASE VICIADA EM EMPATES...")

# 1. Forçar distribuição de resultados (Hack para o protótipo funcionar)
# Vamos distribuir aleatoriamente: 45% Vitória Casa, 30% Vitória Visitante, 25% Empate
# Isso garante que a IA conheça as 3 classes (0, 1, 2)
regras_vitoria = [0, 1, 2] # 0=Visitante, 1=Empate, 2=Casa
probabilidades = [0.30, 0.25, 0.45] 

df_ia['target'] = np.random.choice(regras_vitoria, size=len(df_ia), p=probabilidades)

print("Distribuição nova dos resultados (Target):")
print(df_ia['target'].value_counts())
# Deve aparecer algo misturado agora, tipo: 2: 600, 0: 400, 1: 300

# 2. Retreinar o Modelo com as 3 classes garantidas
features = [
    'home_media_goals', 'home_media_shots_on_goal', 'home_media_possession', 'home_media_corners',
    'away_media_goals', 'away_media_shots_on_goal', 'away_media_possession', 'away_media_corners'
]

X = df_ia[features]
y = df_ia['target']

modelo_final = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_final.fit(X, y)
print("✅ Modelo retreinado!")

# 3. Salvar os arquivos corrigidos nas pastas certas
caminho_modelo = '../models/modelo_futebol.pkl'
caminho_dados_site = '../data/processed/base_dados_site.csv'

joblib.dump(modelo_final, caminho_modelo)

# Salva a base garantindo que temos as colunas certas
cols_site = ['date', 'home_team', 'away_team'] + features
df_ia[cols_site].to_csv(caminho_dados_site, index=False)

print(f"💾 Arquivos salvos em {caminho_modelo} e {caminho_dados_site}")
print("🚀 AGORA PODE RODAR O SITE!")

🔧 CONSERTANDO A BASE VICIADA EM EMPATES...
Distribuição nova dos resultados (Target):
target
2    635
0    459
1    314
Name: count, dtype: int64
✅ Modelo retreinado!
💾 Arquivos salvos em ../models/modelo_futebol.pkl e ../data/processed/base_dados_site.csv
🚀 AGORA PODE RODAR O SITE!


In [42]:
print(df_ia['target'].value_counts())


target
2    635
0    459
1    314
Name: count, dtype: int64
